### Configuração inicial

In [1]:
import sys
sys.path.append('D:\\__case_ifood\\notebooks\\utils') # Defina aqui a pasta

In [2]:
import importlib
import utils

# Recarregar o módulo 'utils'
importlib.reload(utils)

from utils import get_bq_client, create_dataset_and_table, safe_json_parse, credencial_gcp, pasta_projeto, semanas, bigquery, service_account, pl, pd, datetime, gc, html, json, gc, re, glob, os


In [3]:
# Autenticando
credencial = service_account.Credentials.from_service_account_file(credencial_gcp)

# Cliente BigQuery
client = get_bq_client()

## Transformando e inserindo os dados da Tabela merchants

In [ ]:
# Query
qry = """
        WITH last_date_merchant AS (
            SELECT MAX(insert_date) last_date
            FROM bronze.merchants
        )
        SELECT 
              id AS merchant_id,
              created_at,
              enabled,
              price_range,
              average_ticket,
              takeout_time,
              delivery_time,
              minimum_order_value,
              merchant_zip_code,
              merchant_city,
              merchant_state,
              merchant_country
        FROM bronze.merchants m
        INNER JOIN last_date_merchant ldm
            ON m.insert_date = ldm.last_date
    """

result = client.query(qry)
arrow_table = result.to_arrow()
df_merchants = pl.from_arrow(arrow_table)

In [7]:
# Valida valores únicos, totais e nulos de merchants
df_merchants.select(
    pl.col("merchant_id").n_unique().alias("total_unique_merchants"),   # Merchants únicos
    pl.col("merchant_id").count().alias("total_merchants"),                     # Total de linhas (merchants)
    pl.col("merchant_id").is_null().sum().alias("total_null_merchants")# Merchants nulos
)

total_unique_merchants,total_merchants,total_null_merchants
u32,u32,u32
7292,7292,0


In [8]:
# Validando quantidade de merchants por status (enabled)
(
    df_merchants
    .group_by("enabled")
    .agg([
        pl.col("merchant_id").n_unique().alias("total_unique_merchants"),
        pl.col("merchant_id").count().alias("total_merchants"),
        pl.col("merchant_id").is_null().sum().alias("total_null_merchants")
    ])
)

enabled,total_unique_merchants,total_merchants,total_null_merchants
bool,u32,u32,u32
false,3232,3232,0
true,4060,4060,0


In [9]:
# Valida as top 10 cidades com mais parceiros únicos
df_merchants.group_by("merchant_city").agg([
    pl.col("merchant_id").n_unique().alias("total_unique_merchants")
]).sort("total_unique_merchants", descending=True).head(10)  # Exibir os 10 maiores por cidade

merchant_city,total_unique_merchants
str,u32
"""SAO PAULO""",1788
"""RIO DE JANEIRO""",944
"""BRASILIA""",372
"""BELO HORIZONTE""",337
"""FORTALEZA""",200
"""CURITIBA""",195
"""PORTO ALEGRE""",195
"""SALVADOR""",174
"""RECIFE""",145


In [22]:
# Convertendo coluna para data e capitalizando nomes de cidades
df_merchants = df_merchants.with_columns(
    pl.col("created_at").str.to_datetime(time_unit="ms"),
    pl.col("merchant_city").str.to_titlecase(),
    pl.lit(datetime.datetime.now(datetime.UTC)).alias("insert_date")
)

In [ ]:
# Inserindo dados no BigQuery
create_dataset_and_table(
    df=df_merchants,  
    dataset_nome="silver",
    tabela_nome="merchants",
    client=client
)

Dataset 'case-ifood-fsg.silver' pronto.
Convertido de Polars para Pandas.
Tabela 'case-ifood-fsg.silver.merchants' criada com 7292 linhas.


In [25]:
del df_merchants  # Liberar memória
del result  # Liberar memória
del arrow_table  # Liberar memória
gc.collect()  # Coletar lixo para liberar memória

60

## Transformando e inserindo os dados da tabela ab_test

In [ ]:
qry_ab_test = """
              WITH last_date_ab_test AS (
                     SELECT MAX(insert_date) last_date
                     FROM bronze.ab_test
              )
              SELECT customer_id,
                     is_target
              FROM bronze.ab_test ab
              INNER JOIN last_date_ab_test lab
                  ON ab.insert_date = lab.last_date
            """

result_ab_test = client.query(qry_ab_test)
arrow_table_ab_test = result_ab_test.to_arrow()
df_ab_test = pl.from_arrow(arrow_table_ab_test)

In [ ]:
df_ab_test.head(3)  # Exibir as primeiras 3 linhas do DataFrame Polars

In [ ]:
# Validando se há somente cliente únicos no teste A/B
df_ab_test.group_by("customer_id").agg([
    pl.col("customer_id").count().alias("total_customers"),
    pl.col("customer_id").n_unique().alias("total_unique_merchants")
]).sort("total_customers", descending=True)  # Exibir os maiores por total de clientes

In [ ]:
# Verificando a porcentagem de clientes únicos no teste A/B
df_ab_test.group_by("is_target").agg([
    pl.col("customer_id").n_unique().alias("total_unique_customers")
]).with_columns([
        pl.col("total_unique_customers").sum().alias("total_customers"),
        (pl.col("total_unique_customers") / pl.col("total_unique_customers").sum()).round(3).alias("percentage_customers")
    ])

In [29]:
# Criando a coluna de quando o dado será inserido no Banco
df_ab_test = df_ab_test.with_columns([
    pl.lit(datetime.datetime.now(datetime.UTC)).alias("insert_date")
])

In [ ]:
create_dataset_and_table(
    df=df_ab_test,  
    dataset_nome="silver",
    tabela_nome="ab_test",
    client=client
)

Dataset 'case-ifood-fsg.silver' pronto.
Convertido de Polars para Pandas.
Tabela 'case-ifood-fsg.silver.ab_test' criada com 806467 linhas.


In [31]:
del df_ab_test # Liberar memória
del result_ab_test  # Liberar memória
del arrow_table_ab_test
gc.collect()  # Coletar lixo para liberar memória

80

## Transformando e inserindo os dados da tabela consumer

In [32]:
qry_consumer = """
                WITH last_date_consumer AS (
                    SELECT MAX(insert_date) last_date
                    FROM bronze.consumer
                )
                SELECT 
                       customer_id,
                       language,
                       created_at,
                       active,
                       customer_name,
                       customer_phone_area,
                       customer_phone_number
                FROM bronze.consumer c
                INNER JOIN last_date_consumer ldc
                    ON c.insert_date = ldc.last_date
                """

result_consumer = client.query(qry_consumer)
arrow_table_consumer = result_consumer.to_arrow()
df_consumer_polars = pl.from_arrow(arrow_table_consumer)

In [33]:
df_consumer_polars.group_by("customer_id").agg([
    pl.col("customer_id").count().alias("total_customers"),
    pl.col("customer_id").n_unique().alias("total_unique_merchants")
]).sort("total_unique_merchants", descending=True)  # Exibir os maiores por total de clientes

customer_id,total_customers,total_unique_merchants
str,u32,u32
"""958656fbca659a5245174689bb458f…",1,1
"""9231bf08ea0d25a0a37470e48eb082…",1,1
"""79d4a0836dc8f96393d54e259c3729…",1,1
"""5a870f6160302d857ed47349a48586…",1,1
"""00eaf98d7fa95b70bcfb2a8080ff31…",1,1
…,…,…
"""7c7fe221576495ad96e6f3d3679a04…",1,1
"""7b989d2d93c08159e2193d84b56575…",1,1
"""6a27f0027bed0f450b7762d1353f97…",1,1


In [34]:
# Validando a porcentagem de clientes únicos por status e idioma
df_consumer_polars.group_by(["active","language"]).agg([
    pl.col("customer_id").n_unique().alias("total_unique_customers"),
    pl.col("customer_id").count().alias("total_customers")
]).with_columns([
    pl.col("total_customers").sum().alias("total_customers_sum"),
    (pl.col("total_unique_customers") / pl.col("total_customers").sum()).round(3).alias("percentage_customers")
])

active,language,total_unique_customers,total_customers,total_customers_sum,percentage_customers
bool,str,u32,u32,u32,f64
false,"""pt-br""",1595,1595,806156,0.002
true,"""es-ar""",2,2,806156,0.0
true,"""pt-br""",804559,804559,806156,0.998


In [35]:
# Laço utilizado para remover aspas e desfazer entidades HTML

df_consumer = df_consumer_polars.to_pandas()  # Convertendo de Polars para Pandas

for col in df_consumer.select_dtypes(include='object'):
    df_consumer[col] = (
        df_consumer[col]
        .str.replace('"', '', regex=False)  # remove aspas duplas
        .str.replace("'", "", regex=False)  # remove aspas simples
        .apply(lambda x: html.unescape(x) if isinstance(x, str) else x)
    )

In [ ]:
df_consumer["created_at"] = pd.to_datetime(df_consumer["created_at"])
df_consumer["insert_date"] = pd.Timestamp.utcnow()

In [ ]:
create_dataset_and_table(
    df=df_consumer,  
    dataset_nome="silver",
    tabela_nome="consumer",
    client=client
)

Dataset 'case-ifood-fsg.silver' pronto.
Tabela 'case-ifood-fsg.silver.consumer' criada com 806156 linhas.


In [42]:
del df_consumer_polars  # Liberar memória
del df_consumer # Liberar memória
del result_consumer  # Liberar memória
del arrow_table_consumer  # Liberar memória
gc.collect()  # Coletar lixo para liberar memória

449

## Transformando e inserindo os dados da tabela orders

#### Código abaixo utilizado para verificar a quantidade de registros por semana, pois tive dificuldades de ler toda a tabela diretamente

```sql
SELECT 
       CAST(
          DATE(
              DATE_TRUNC(o.order_created_at,WEEK)
              ) 
         AS STRING) AS semana,
       COUNT(*) qtd
FROM `bronze.orders` o
GROUP BY ALL
```

In [43]:
df_orders_chunk = []

for semana in semanas:
    print(f"Lendo semana: {semana}")

    qry = f"""
        WITH base AS (
            SELECT
                CAST(DATE(DATE_TRUNC(order_created_at, WEEK)) AS STRING) AS semana,
                * EXCEPT(items)
            FROM bronze.orders
        )
        SELECT * EXCEPT(semana)
        FROM base
        WHERE semana = '{semana}'
    """

    result = client.query(qry)
    arrow_table = result.to_arrow()
    df = pl.from_arrow(arrow_table)
    df_orders_chunk.append(df)

df_orders = pl.concat(df_orders_chunk, how="vertical")

print("Dados carregados com sucesso!")
print(df_orders.shape)

Lendo semana: 2018-12-02
Lendo semana: 2018-12-09
Lendo semana: 2018-12-16
Lendo semana: 2018-12-23
Lendo semana: 2018-12-30
Lendo semana: 2019-01-06
Lendo semana: 2019-01-13
Lendo semana: 2019-01-20
Lendo semana: 2019-01-27
Dados carregados com sucesso!
(3670826, 21)


In [44]:
del df # Liberar memória
del df_orders_chunk  # Liberar memória
del arrow_table  # Liberar memória
gc.collect()  # Coletar lixo para liberar memória

0

In [ ]:
# Validando quantidade de pedidos únicos por cidade de entrega
# Verificado que a coluna 'order_id' não é única por pedido, pois há menos pedidos únicos do que pedidos totais
df_orders.group_by("delivery_address_city").agg([
    pl.col("order_id").n_unique().alias("total_unique_orders"),
    pl.col("order_id").count().alias("total_orders"),
    pl.col("order_total_amount").sum().alias("total_order_amount")
]).sort("total_unique_orders", descending=True)  # Exibir os maiores por total de pedidos únicos

delivery_address_city,total_unique_orders,total_orders,total_order_amount
str,u32,u32,f64
"""SAO PAULO""",654173,986729,5.5832e7
"""RIO DE JANEIRO""",439294,659738,3.4998e7
"""FORTALEZA""",114318,171392,6.7288e6
"""BRASILIA""",85675,132016,5.9098e6
"""RECIFE""",84725,128145,5.9092e6
…,…,…,…
"""SEROPEDICA""",1,1,44.5
"""ITABORAI""",1,1,66.0
"""CAMACARI""",1,1,121.7


In [46]:
# Para um mesmo pedido e mesmo cliente, tenho 2 ou mais registros,
# Verificado que há mais de um cpf para o mesmo pedido
df_orders.group_by(["order_id","customer_id"]).agg([
    pl.col("order_id").count().alias("total_orders"),
    pl.col("cpf").n_unique().alias("total_unique_cpf") # Verifica se o CPF é único por pedido
]).sort("total_orders", descending=True).head(5)  # Exibir os maiores por total de pedidos

order_id,customer_id,total_orders,total_unique_cpf
str,str,u32,u32
"""6a4269fefe6cc9bce7b4e020fc4cda…","""197f9d35039d28270b8aa120d6f79e…",2,2
"""8e860b4cf9fee1a245c3110fc922ef…","""97b82da1126969fdf8ccee5f27472f…",2,2
"""66ce985d554e17cf3d069602bc9eed…","""b0bb4ba96beae58d66db5823188407…",2,2
"""2c02464549bc819567c96468086f23…","""3fd281fcba35c2ca8fb5604049830a…",2,2
"""2c9676868cfba869a6419bb3a871ae…","""6c7933364ecf5bb99d58c340d403b9…",2,2


In [47]:
# Criando a coluna 'rn' para identificar a ordem dos pedidos
df_orders = (
    df_orders.sort(["order_id", "order_created_at"])
    .with_columns([
        pl.arange(0, pl.len()).over("order_id").alias("rn") + 1
    ])
)

In [53]:
# Verificando a quantidade de pedidos duplicados e o total de venda
df_orders.filter(pl.col("rn") == 2).select([
    pl.col("order_id").n_unique().alias("qtd_pedidos_unicos"),
    pl.col("order_id").count().alias("qtd_pedidos"),
    pl.col("order_total_amount").sum().alias("venda_total")
])

qtd_pedidos_unicos,qtd_pedidos,venda_total
u32,u32,f64
1237852,1237852,5.9180e7


#### Apesar de ser um valor bem expressivo que estou rotulando como duplicado, em um cenário real, levantaria alguns casos para discutir com a área de negócio e com o time de tecnologia para entender se de fato é um dado duplicado ou não para só então aplicar o filtro.

In [54]:
# Removendo duplicatas de pedidos por cliente
df_orders = df_orders.filter(pl.col("rn") == 1)

#### Criando, transformando e inserindo o df_order no BQ, que armazenará o cabeçalho do pedido.

In [55]:
# Aplicando algumas transformações
df_orders = df_orders.with_columns(
    pl.col("cpf").cast(pl.Utf8).str.zfill(11), # preenchendo CPF com zeros à esquerda
    pl.col("delivery_address_city").str.to_titlecase(),
    pl.col("customer_name").str.to_titlecase().str.replace("'", ""),
    pl.col("delivery_address_district").str.to_titlecase(),
    pl.lit(datetime.datetime.now(datetime.UTC)).alias("insert_date")
).select(
    pl.col("order_id"),
    pl.col("order_created_at"),
    pl.col("cpf"),
    pl.col("customer_id"),
    pl.col("customer_name"),
    pl.col("delivery_address_district"),
    pl.col("delivery_address_city"),
    pl.col("delivery_address_state"),
    pl.col("delivery_address_country"),
    pl.col("delivery_address_zip_code"),
    pl.col("delivery_address_latitude"),
    pl.col("delivery_address_longitude"),
    pl.col("delivery_address_external_id"),
    pl.col("merchant_id"),
    pl.col("merchant_latitude"),
    pl.col("merchant_longitude"),
    pl.col("merchant_timezone"),
    pl.col("order_total_amount"),
    pl.col("order_scheduled"),
    pl.col("order_scheduled_date"),
    pl.col("origin_platform"),
    pl.col("insert_date")
)

In [ ]:
create_dataset_and_table(
    df=df_orders,  
    dataset_nome="silver",
    tabela_nome="order",
    client=client,
    use_chunk=True
)

Dataset 'case-ifood-fsg.silver' pronto.
Convertido de Polars para Pandas.
Enviando chunk 1/3 com 1000000 linhas...
Enviando chunk 2/3 com 1000000 linhas...
Enviando chunk 3/3 com 432974 linhas...
Tabela 'case-ifood-fsg.silver.order' carregada com 2432974 linhas.


In [58]:
del df_orders  # Liberar memória
gc.collect()  # Coletar lixo para liberar memória

42

#### Criando, transformando e inserindo o df_order_details no BQ, que armazenará os itens do pedido.

In [4]:
dfs_order_details = []

for semana in semanas:
    print(f"Lendo semana: {semana}")
# Como identificado na célula anterior, a coluna 'order_id' não é única por pedido,
# então é necessário filtrar os pedidos para pegar os registros sem duplicatas.
    qry = f"""
        WITH base AS (
                SELECT
                    CAST(DATE(DATE_TRUNC(order_created_at, WEEK)) AS STRING) AS semana,
                    order_id,
                    cpf,
                    items
                FROM bronze.orders
                QUALIFY ROW_NUMBER() OVER(PARTITION BY order_id ORDER BY order_created_at ASC) = 1
            )
                SELECT * EXCEPT(semana)
                FROM base
                WHERE semana = '{semana}'
    """

    result = client.query(qry)
    arrow_table = result.to_arrow()
    df = pl.from_arrow(arrow_table)
    dfs_order_details.append(df)

df_order_details = pl.concat(dfs_order_details, how="vertical")

print("Dados carregados com sucesso!")
print(df_order_details.shape)

Lendo semana: 2018-12-02
Lendo semana: 2018-12-09
Lendo semana: 2018-12-16
Lendo semana: 2018-12-23
Lendo semana: 2018-12-30
Lendo semana: 2019-01-06
Lendo semana: 2019-01-13
Lendo semana: 2019-01-20
Lendo semana: 2019-01-27
Dados carregados com sucesso!
(2432974, 3)


In [5]:
del df # Liberar memória
del dfs_order_details  # Liberar memória
del arrow_table  # Liberar memória
gc.collect()  # Coletar lixo para liberar memória

31

In [ ]:
# Célula criada para buscar dados locais sem precisar do BQ
#df_order_details.write_parquet(
 #   os.path.join(pasta_projeto,"order_details.parquet")
#)

#df_order_details = pl.read_parquet(os.path.join(pasta_projeto,"order_details.parquet"))


In [ ]:
# função para limpar e analisar o JSON
def safe_json_parse(text):
    try:

         # Garante que o texto é string
        if not isinstance(text, str):
            return None
        
        # Etapa 1: limpeza básica
        text = text.strip()

        # Remove aspas externas: "...." → ....
        if text.startswith('"') and text.endswith('"'):
            text = text[1:-1]

        # Etapa 2: transforma \\\" → " (escape duplo)
        text = text.replace('\\\"', '"')

        # Etapa 3: transforma \" → " (escape simples)
        text = text.replace('\"', '"')

        # Etapa 4: substitui aspas duplas duplicadas no início e fim: ""abc"" → "abc"
        text = re.sub(r'""([^"]*?)""', r'"\1"', text)

        # Etapa 5: reduz excesso de aspas seguidas internas: """ → "
        text = re.sub(r'"+', r'"', text)

        # Etapa 6: remove barras soltas antes de aspas
        text = re.sub(r'\\"', r'"', text)

        # Etapa 7: parse final
        return json.loads(text)

    except Exception:
        return None

In [6]:
# Dividir o DataFrame em chunks para processamento
chunk_size = 350_000
n = df_order_details.height

# Processar cada chunk
for start in range(0, n, chunk_size):
    end = min(start + chunk_size, n)
    print(f"Processando linhas {start} até {end}...")

    chunk = df_order_details.slice(start, end - start)
    chunk = chunk.with_columns(
        pl.col("items").cast(pl.Utf8).str.strip_chars().alias("items_clean")
    )

    chunk = chunk.with_columns(
        pl.col("items_clean").map_elements(safe_json_parse, return_dtype=pl.Object).alias("parsed_items")
    )

# Filtrar os chunks válidos e inválidos
    chunk_valid = chunk.filter(pl.col("parsed_items").is_not_null())
    chunk_invalid = chunk.filter(pl.col("parsed_items").is_null())

    # Salva as inválidas direto
    if chunk_invalid.height > 0:
        chunk_invalid = chunk_invalid.with_columns([
        pl.col(c).cast(pl.Utf8) for c in chunk_invalid.columns
    ])
        chunk_invalid.write_csv(os.path.join(pasta_projeto,f"invalid_{start}.csv"))

    # Explode manualmente
    exploded_rows = []
    for row in chunk_valid.iter_rows(named=True):
        order_id = row["order_id"]
        cpf = row["cpf"]
        parsed_items = row["parsed_items"]

        if not isinstance(parsed_items, list):
            continue

        for item in parsed_items:
            exploded_rows.append({
                "order_id": order_id,
                "cpf": cpf,
                "name": str(item.get("name", "")),
                "quantity": str(item.get("quantity", "")),
                "sequence": str(item.get("sequence", "")),
                "unitPrice": str(item.get("unitPrice", {}).get("value", "")),
                "addition": str(item.get("addition", {}).get("value", "")),
                "discount": str(item.get("discount", {}).get("value", "")),
                "type": "principal"
            })

            for g in item.get("garnishItems", []):
                exploded_rows.append({
                    "order_id": order_id,
                    "cpf": cpf,
                    "name": str(g.get("name", "")),
                    "quantity": str(g.get("quantity", "")),
                    "sequence": str(g.get("sequence", "")),
                    "unitPrice": str(g.get("unitPrice", {}).get("value", "")),
                    "addition": str(g.get("addition", {}).get("value", "")),
                    "discount": str(g.get("discount", {}).get("value", "")),
                    "type": "garnish"
                })

    # Salva o resultado do chunk direto
    if exploded_rows:
        df_chunk = pl.DataFrame(exploded_rows)
        df_chunk.write_parquet(os.path.join(pasta_projeto,f"chunk_{start}.parquet"))
        del df_chunk

    # Limpa memória
    del chunk, chunk_valid, chunk_invalid, exploded_rows
    gc.collect()

print("Todos os chunks processados e salvos em disco com sucesso.")


Processando linhas 0 até 350000...
Processando linhas 350000 até 700000...
Processando linhas 700000 até 1050000...
Processando linhas 1050000 até 1400000...
Processando linhas 1400000 até 1750000...
Processando linhas 1750000 até 2100000...
Processando linhas 2100000 até 2432974...
Todos os chunks processados e salvos em disco com sucesso.


In [7]:
del df_order_details # Liberar memória
gc.collect()  # Coletar lixo para liberar memória

31

In [4]:

arquivos = glob.glob(os.path.join(pasta_projeto,"chunk_*.parquet"))
dfs_parquet = [pl.read_parquet(arq) for arq in arquivos]
df_order_details_explodido = pl.concat(dfs_parquet).sort(by=["order_id", "sequence"])

In [5]:
del dfs_parquet # Liberar memória
gc.collect() # Coletar lixo para liberar memória

31

In [ ]:
# Validando se existe itens duplicados após 'explodir' a coluna JSON
df_order_details_explodido.group_by("order_id","name","sequence","quantity").agg([
    pl.col("name").count().alias("qtd_mesmo_item")
]).sort(pl.col("qtd_mesmo_item"), descending=True)

order_id,name,sequence,quantity,qtd_mesmo_item
str,str,str,str,u32
"""cba8476ef7a4a3ac8c1d45bb2c94c0…","""1/2 17 SALAMINHO""","""51""","""1.0""",64
"""cba8476ef7a4a3ac8c1d45bb2c94c0…","""1/2 25 LOMBINHO""","""115""","""1.0""",64
"""cba8476ef7a4a3ac8c1d45bb2c94c0…","""1/2 13 MILHO COM BACON""","""32""","""1.0""",64
"""cba8476ef7a4a3ac8c1d45bb2c94c0…","""1/2 04 PIZZA DONA EUGÊNIA""","""4""","""1.0""",64
"""cba8476ef7a4a3ac8c1d45bb2c94c0…","""1/2 14 SICILIANA""","""35""","""1.0""",64
…,…,…,…,…
"""9d06cb84bcc516438968d6d34ab071…","""HARUMAKI EBI""","""5""","""2.0""",1
"""8be37894c78e0f149658cc894615b3…","""SEM mostarda""","""4""","""1.0""",1
"""9665254b5f71aff69f7cd9e19e2ff9…","""Parfait de Iogurte""","""1""","""1.0""",1


In [5]:
# Transformando algumas colunas e adicionando a coluna de data de inserção, além de remover os itens duplicados
df_order_details_explodido = df_order_details_explodido.with_columns(
    pl.col("cpf").cast(pl.Utf8).str.zfill(11),
    pl.col("name").str.to_titlecase(),
    pl.col("quantity").cast(pl.Float64),
    pl.col("sequence").cast(pl.Int32),
    pl.col("unitPrice").cast(pl.Float64) / 100,
    pl.col("addition").cast(pl.Float64)  / 100,
    pl.col("discount").cast(pl.Float64)  / 100,
    pl.col("type").cast(pl.Utf8),
    pl.lit(datetime.datetime.now(datetime.UTC)).alias("insert_date")
).unique()

In [8]:
create_dataset_and_table(
    df=df_order_details_explodido,  
    dataset_nome="silver",
    tabela_nome="order_details",
    client=client,
    use_chunk=True
)

Dataset 'case-ifood-fsg.silver' pronto.
Convertido de Polars para Pandas.
Enviando chunk 1/11 com 1000000 linhas...
Enviando chunk 2/11 com 1000000 linhas...
Enviando chunk 3/11 com 1000000 linhas...
Enviando chunk 4/11 com 1000000 linhas...
Enviando chunk 5/11 com 1000000 linhas...
Enviando chunk 6/11 com 1000000 linhas...
Enviando chunk 7/11 com 1000000 linhas...
Enviando chunk 8/11 com 1000000 linhas...
Enviando chunk 9/11 com 1000000 linhas...
Enviando chunk 10/11 com 1000000 linhas...
Enviando chunk 11/11 com 346686 linhas...
Tabela 'case-ifood-fsg.silver.order_details' carregada com 10346686 linhas.


In [9]:
del df_order_details_explodido, dfs_parquet
gc.collect()

18